###Project by Srinath Sai Tripuraneni (A20338902).

##Initial process of making M into file and r_old into file

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #importing drive folder.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Note :- Here I have uploaded the given 2 folders in my drive , and took that path ,for execution you need to specify your folder path , this is the initial process of constructing M file which takes some time and this time is not considered in execution time.

In [ ]:
#folder = ''
import os
import re
#articles_list = []
M_matrix=open('M.txt','w') #opening new empty M file.
#r_old_initial=open('r_old','w')
for root, dirs, files in os.walk("/content/drive/MyDrive/Dataset"): #iteraing over directory
    for file in files: #iterating per file,because here every file is an article instead of combining all files to a single file thaught to iterate over file by file which is article by article for memory efficiency.
        if file.endswith('.txt'): #though all are text files this is just an block to perform cleaning operation.
            with open(os.path.join(root, file), 'r') as f: #opening every article
                text = f.read() #reading every article matter in string format.
                y=text.split('</title') #as source of page is between <title> and </title>, just splitted on </title> which makes entire article matter into 2 space array in which 1st index will have <title> source 
                #next index will have rest of article matter excluding </title>
                z=y[0].split('<title>') #now again spliting first index of list on <title> which will give source in z[1] place.
                each_row=() #taking tuple which will stores first element as source, second element is degree and third element is list of destinations.
                m=[m1.start() for m1 in re.finditer('\[\[', y[1])] # as our destinations are in between [[ ]], first finding indices of [[ and storing them in list m.
                n=[n1.start() for n1 in re.finditer('\]\]', y[1])] # using re.finditer finding indices of only ]] and storing in list n.
                destinations=[] #list of destinations to extract strings between indices in list m and list n.
                if((len(m)>0 and len(n)>0) and (len(m)==len(n))): #this is important step as some articles may contain only ]] or may contain only [[ which are not required.
                  for i in range(0,len(m)):# after filtering we will proceed as [[ ]] these lists length should be equal.
                    if(y[1][m[i]+2:n[i]] not in destinations): #now extracting strings between [[ ]] by substring from main string of matter article, using indices stored in m and n.
                      destinations.append(y[1][m[i]+2:n[i]]) # appending destinations extracted in [[ ]].
                each_row=(z[1],len(destinations),destinations) # appending in tuple as source, degree (len of list of destinations), destinations list.
                if(each_row[1]>0): # eleminating the sources which are having zero degrees.
                  M_matrix.write(str(each_row[0]) + '||#' + str(each_row[1]) + '||#' +  str(each_row[2])) # writing into M file.
                  M_matrix.write("\n") # for not concatenating with next line.
                  #r_old_initial.write(str(each_row[0]) + ':-' + )
M_matrix.close()
                  
#FINALLY AFTER THIS ITERATION MY M.txt WILL HAVE LINES OF SOURCES IN THE FORMAT 'SOURCE'||#'DEGREE'||#'DESTINATIONS' . I PURPOSEDLY WRITTEN INTO FILE LIKE THIS SO THAT IN ALGORITHM I CAN
#DIRECTLY SPLIT ON '||#'.




In [ ]:
with open(r"M.txt", 'r') as fp:
  N = len(fp.readlines()) #getting N value for the formula.
  print('Total Number of Page sources in M without zero degree:', N)


Total Number of Page sources in M without zero degree: 38302


In [ ]:
beta=0.8 #given beta value.
r_new={} #storing r_new hashmap which is allowed and why hashmap because for specific destination updation must be done right so keys will be graph nodes and values will be page ranks

In [ ]:
#CREATING INTIAL r_old FILE WITH EVERY SOURCE AND  1/N PAGE RANK VALUE.
r_old_initial=open('r_old','w') #intialing r_old empty file for writing inital r old.
with open('M.txt') as f:
    for line in f:
        line_cache=line.split('||#') #from M file for every line, splitting on ||# which will give us three spaced array in which first space is source , second space will be degree and third space will be list of destinations.
        r_old_initial.write(str(line_cache[0]) + ':-' + str(1/N)) #for every source writing as source :- 1/N 
        r_old_initial.write("\n") # for not concatenating next line.
        #r_new[line_cache[0]]=((1-beta)/N)
r_old_initial.close()

#SO FINALLY r_old FILE WILL HAVE ALL NODES AS NODE :- and 1/N VALUE RESPECTIVELY.

In [ ]:
with open('r_old') as f:
  for line in f:           
    rank_cache=line.split(':-')   # now from r_old file initializing r_new for every line I will split based on :- to get nodes.
    r_new[rank_cache[0]]=((1-beta)/N) #storing in hashmap with respective node as key and (1-beta)/N

##Algorithm implementation

In [ ]:
from ast import literal_eval #necessary package for extracting destinations in algorithm.

In [ ]:
import time #time module has been exported.


In [ ]:
start_time = time.time() #time of execution starts here.
for i in range(1,16): # 15 iterations of entire algorithm.
  #sum=0
  #here r_new have all sources right, so for every page in first line of algorithm means iterating over hashmap which is only in memory.
  for x in r_new: # for every source according to algorithm we should read M file for destinations, degree and read r_old file for respective value;
    destinations=[]
    degree=0 #intializing all required variables for every source. before opening M file and r_old File
    r_old_value=0.000000
    with open(r"M.txt",'r') as f: #Here M file has been opened.
      for line_number,line in enumerate(f): #iterating line by line of file instead of storing all mater for memory efficency.
        line_cache=line.split('||#') #every line will be splitted on ||# for every iteration.
        if(x==line_cache[0]): # if the page source x in r_new is same as source in M
          degree=int(line_cache[1]) # taking degree of that source
          destinations=literal_eval(line_cache[2]) #literal_eval is the python package which will extract without changing datastructure format from string. So we get destinations list.
          break # when we get destinations and degree for one page breaking the loop of scanning of file for time efficiency.
    with open(r"r_old",'r') as f: #scanning of r_old file for respective r_old value.
      for line_number,line in enumerate(f): #reading as for each line instead of storing entire matter for memory efficiency.
        r_old_cache=line.split(':-') #splitting on :-
        if(r_old_cache[0]==x): #again if we get any source equals to source in r_old file 
          m = re.findall(r'\d+\.\d+', r_old_cache[1]) #this will give you digits in value from r_old matching line and from second element after splitting on ':-'
          r_old_value=float(m[0]) #taking into float of respective value of matched source.
          break #breaking the scan of file after getting respective page rank value for time efficiency.
   
    for destination in destinations:
      if(destination in r_new):
        for j in range(1,degree+1):                            #last part of algorithm after getting every page's old rank value, degree, destinations.
          r_new[destination]=beta*(r_old_value/degree) +(1-beta)/N #appending new ranks with this formula.

    sum=0
    for x in r_new:
      sum+=r_new[x] #taking sum of all pageranks after updating.
    for x in r_new:
      r_new[x]+=(1-sum)/N #again updating for all pageranks and this ensures less leaking pages.
    r_new1=open('r_old','w')
    for y in r_new:
      r_new1.write(str(y) + ':-' + str(r_new[y])) # this final step for every iteration in which we overwrite r_new values to r_old in r_old file. . This is exacltly doing r_new to r_old after every iteration.
      r_new1.write("\n") #at the end of each iteration r_new is similary written as same format as r_old.
    r_new1.close()

end_time=time.time()
  #end time will be calculated after each iteration.





In [ ]:
print('execution time is :-' + str(end_time-start_time)) #execution time of my implementation in seconds.

execution time is :-63509.9616894722


##Validity of my implementation.

Generally all page ranks sum should be 1 ideally, but in our graph there are some nodes which can leak and reduce this rank but very slightly.

In [ ]:
sum=0;
for x in r_new: #summing of all page ranks in stored hashmap
  sum+=r_new[x]
print(sum) # this sums show that my algorithm implementation is valid as because after 15 iterations it reduced from 1 by very very less significant amount by 1.

0.9997553395888826


##Output

In [ ]:
from collections import OrderedDict #package for arranging values based on order.
descending_page_ranks = OrderedDict(sorted(r_new.items(), key=lambda x: x[1], reverse=True)) #from r_newsorting based on decending page ranks.
output=[] # output list for display.
for x in descending_page_ranks:
  output.append((x,descending_page_ranks[x]/1000)) # here every page value is very very less which can have something as 3.something *e^-5 something. but printing as string format is allowed only taking number not e^-5 thing ,
  #since all ranks sum must be 0.999755339588826 from above to not look awkward divided by 1000 but I checked every value of the ranks and observed to be *e^-5 something which justifies my validity algorithm.
for i in range(0,25): # printing top 25 pages with their ranks.
  print(str(output[i][0]) + ":" , output[i][1])

James Edward Keeler: 0.007904241858745437
Duck: 0.007831784025668052
Category:Social sciences: 0.007536642960715786
Wallbach, Switzerland: 0.007485269058668212
Fremantle, Western Australia: 0.007452869784595176
Mary, Crown Princess of Denmark: 0.007337437524636131
Sendero Luminoso: 0.00701641185389771
Eastern long-beaked echidna: 0.006885054355985402
Continental drift: 0.006702555004102609
2004 Redfern riots: 0.006693185479334484
Jester: 0.0066568727881050034
Becker (TV series): 0.006522376225418934
Sibling: 0.006378160679742978
Virus: 0.0063163204500442955
Western long-beaked echidna: 0.006241108373065668
Salamander: 0.006188671101753893
Psychokinesis: 0.006106585179142002
May 1: 0.00591527877121967
Tyndareus: 0.005779078786611759
Christoph Willibald von Gluck: 0.005700219173290185
Union for a Popular Movement: 0.005682415784185709
Sanjo Station (Kyoto): 0.0056181284141373685
Twin town: 0.005580782777799577
Tour of Britain: 0.0055072167056761465
Arachnid: 0.005479772816033096
